---
## 1. AI_SENTIMENT: Analyze Product Review Sentiment

AI_SENTIMENT returns overall and category-specific sentiment analysis. You can optionally specify categories (up to 10) to extract sentiment for specific aspects.

In [ ]:
-- Basic sentiment analysis (overall only)
SELECT 
  product_id,
  review_title,
  rating,
  SNOWFLAKE.CORTEX.AI_SENTIMENT(review_text) AS sentiment_result,
  SNOWFLAKE.CORTEX.AI_SENTIMENT(review_text):categories[0]:sentiment::VARCHAR AS overall_sentiment
FROM product_reviews
ORDER BY review_id DESC
LIMIT 10;

### Advanced: Category-Specific Sentiment

Extract sentiment for specific aspects like quality, price, performance, etc.

#### SQL Approach with Categories

In [ ]:
-- Sentiment analysis with specific categories
SELECT 
  product_id,
  review_title,
  rating,
  SNOWFLAKE.CORTEX.AI_SENTIMENT(
    review_text,
    ['quality', 'price', 'performance', 'durability', 'comfort']
  ) AS sentiment_by_category,
  sentiment_by_category:categories[0]:sentiment::VARCHAR AS overall_sentiment,
  sentiment_by_category:categories[1]:sentiment::VARCHAR AS quality_sentiment,
  sentiment_by_category:categories[2]:sentiment::VARCHAR AS price_sentiment,
  sentiment_by_category:categories[3]:sentiment::VARCHAR AS performance_sentiment,
  sentiment_by_category:categories[4]:sentiment::VARCHAR AS durability_sentiment,
  sentiment_by_category:categories[5]:sentiment::VARCHAR AS comfort_sentiment
FROM product_reviews
ORDER BY review_id DESC
LIMIT 10;

---
## 2. AI_AGG: Aggregate and Summarize Reviews by Product

AI_AGG aggregates multiple text values and generates a summary using LLMs.

In [ ]:
SELECT 
  pc.product_name,
  pc.product_category,
  COUNT(pr.review_id) AS total_reviews,
  ROUND(AVG(pr.rating), 2) AS avg_rating,
  SNOWFLAKE.CORTEX.SUMMARIZE(
    LISTAGG(pr.review_text, '\n\n---\n\n')
  ) AS review_summary
FROM product_catalog pc
LEFT JOIN product_reviews pr ON pc.product_id = pr.product_id
WHERE pr.review_text IS NOT NULL
GROUP BY pc.product_name, pc.product_category
ORDER BY avg_rating DESC
LIMIT 5;

---
## 3. AI_COMPLETE: Generate Marketing Content

AI_COMPLETE uses LLMs to generate text based on prompts.

In [ ]:
SELECT 
  product_name,
  product_category,
  price,
  SNOWFLAKE.CORTEX.COMPLETE(
    'llama3.1-70b',
    'Based on this product: ' || product_name || 
    ' in category ' || product_category || 
    ' priced at $' || price || 
    ', write a compelling 2-sentence marketing pitch for winter sports enthusiasts.'
  ) AS marketing_pitch
FROM product_catalog
WHERE product_category IN ('Skis', 'Snowboards')
LIMIT 3;

---
## 4. AI_EXTRACT: Extract Key Information from Reviews

AI_EXTRACT extracts structured information from unstructured text.

In [ ]:
SELECT 
  product_id,
  review_title,
  rating,
  SNOWFLAKE.CORTEX.EXTRACT_ANSWER(
    review_text,
    'What are the main pros and cons mentioned in this review?'
  ) AS extracted_pros_cons
FROM product_reviews
WHERE rating IN (5, 1, 2)
LIMIT 5;

---
## 5. Cortex Search: Semantic Product Search

Query the Cortex Search service for semantic product discovery.

In [ ]:
-- Find products related to "backcountry skiing in deep snow"
SELECT PARSE_JSON(
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
    'automated_intelligence.raw.product_search_service',
    '{
      "query": "backcountry skiing in deep snow",
      "columns": ["product_name", "product_category", "description", "price"],
      "limit": 5
    }'
  )
)['results'] AS search_results;